## BIPARTITE GRAPHS
Here is a dataset that shows a simple 2-node network:  the attendance of 18 Southern Women at 14 social events:

A bipartite graph consists of two groups of nodes. There are links between nodes of differing groups, but no links among nodes from the same group. Common examples are customers and purchased products or meetups and people attending. Which of the groups a node belongs to can be indicated by the keyword "bipartite" and the corresponding group.


### Packages

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import networkx.algorithms.bipartite as bipartite
import numpy as np
import pandas as pd

Davis Southern Women Graph

In [ ]:
G = nx.davis_southern_women_graph()
women = G.graph["top"]
clubs = G.graph["bottom"]
c = bipartite.color(G)

### Network Info

In [ ]:
print(nx.info(G))

The graphs, nodes and edges of the two groups of the bipartite graph can easily be extracted.

In [ ]:
women, clubs = bipartite.sets(G)
print("First Group:", women)
print("Second Group:", clubs)

To represent the bipartite graph, we fix the positions of the nodes according to which group they belong to.

In [ ]:
pos = dict()
pos.update( (n, (1,i)) for i, n in enumerate(women) )
pos.update( (n, (2,i)) for i, n in enumerate(clubs) )
num_nodes = G.number_of_nodes()
cmap = plt.get_cmap('Blues')


### Bipartile Graph 1
By creating this graph we are able to understand the amount of events each women attended.
there are some women that stand out from the rest for the amount of events they attended some more than others

* THeresa
* Katherina
* Nora

In [ ]:
plt.figure(figsize=(15,15))
nx.draw(G,
        pos= pos,
        with_labels=True,
        node_color = np.random.random(num_nodes),
        cmap = plt.get_cmap('Oranges'))
plt.show()

### Relationships
How is the original graph related to the original bipartite graph

In [ ]:
G_top = bipartite.projected_graph(G, women)
G_bottom = bipartite.projected_graph(G, clubs)

In [ ]:
print(G_bottom.edges())

### Women

In [ ]:
nx.draw(G_top, with_labels=True)

### Events

In [ ]:
nx.draw(G_bottom, with_labels=True)

## Centrality Measures
### Betweenness Centrality

In [ ]:
bc = nx.betweenness_centrality(G)
df_bc = pd.DataFrame.from_dict({
        'node': list(bc.keys()),
        'betweenness': list(bc.values())
})
df_bc.sort_values('betweenness', ascending=False)

### Closeness Centrality

In [ ]:
cc = nx.closeness_centrality(G)
df_cc = pd.DataFrame.from_dict({
        'node': list(cc.keys()),
        'closeness': list(cc.values())
})
df_cc.sort_values('closeness', ascending=True)

### Degree Centrality

In [ ]:
dc = nx.degree_centrality(G)
df_dc = pd.DataFrame.from_dict({
        'node': list(dc.keys()),
        'degree': list(dc.values())
})
df_dc.sort_values('degree', ascending=False)

### Summary

In [ ]:
summary_df = pd.concat([df_bc,df_cc,df_dc], axis=1)
summary_df

### Centrality Graphs

In [ ]:
viz = summary_df.plot.bar(rot=90, subplots=True, figsize=(20,10))

## Hierarchical Clustering

In [ ]:
cluster = bipartite.average_clustering(G)
cluster

In [ ]:
cluster_top = bipartite.average_clustering(G, G_top)
cluster_top

In [ ]:
cluster_bottom = bipartite.average_clustering(G, G_bottom)
cluster_bottom

### Block Model
A block model is a simplified network derived from the original network, where all
nodes in a cluster are considered a single node, and all relationships between original
nodes become aggregated into relationships between blocks.


In [ ]:
from collections import defaultdict
from scipy.cluster import hierarchy
from scipy.spatial import distance


def create_hc(G):
        """Creates hierarchical cluster of graph G from distance matrix"""
        path_length = nx.all_pairs_shortest_path_length(G)
        distances = np.zeros((len(G), len(G)))
        for u, p in path_length:
                for v, d in p.items():
                        distances[u][v] = d
        # Create hierarchical cluster
        Y = distance.squareform(distances)
        Z = hierarchy.complete(Y)  # Creates HC using farthest point linkage
        # This partition selection is arbitrary, for illustrive purposes
        membership = list(hierarchy.fcluster(Z, t=1.15))
        # Create collection of lists for blockmodel
        partition = defaultdict(list)
        for n, p in zip(list(range(len(G))), membership):
                partition[p].append(n)
        return list(partition.values())


G = nx.davis_southern_women_graph()

# Extract largest connected component into graph H
H = G.subgraph(next(nx.connected_components(G)))
# Makes life easier to have consecutively labeled integer nodes
H = nx.convert_node_labels_to_integers(H)
# Create parititions with hierarchical clustering
partitions = create_hc(H)
# Build blockmodel graph
BM = nx.quotient_graph(H, partitions, relabel=True)

# Draw original graph
print("Original Graph")
pos = nx.spring_layout(H, iterations=100, seed=83)  # Seed for reproducibility
plt.figure(figsize=(15,15))
plt.subplot(211)
nx.draw(H, pos, with_labels=True, node_size=10)



In [ ]:
# Draw block model with weighted edges and nodes sized by number of internal nodes
node_size = [BM.nodes[x]["nnodes"] * 10 for x in BM.nodes()]
edge_width = [(2 * d["weight"]) for (u, v, d) in BM.edges(data=True)]
# Set positions to mean of positions of internal nodes from original graph
posBM = {}
for n in BM:
        xy = np.array([pos[u] for u in BM.nodes[n]["graph"]])
        posBM[n] = xy.mean(axis=0)

print("Block Model")
plt.figure(figsize=(15,15))
plt.subplot(212)
nx.draw(BM, posBM, node_size=node_size, width=edge_width, with_labels=True)
plt.axis("off")
plt.show()

## Island Method